In [1]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import tree.ctutils as ctu

## fucntions 
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)


def plot_lambda(catalog, i_early, i_late, i_bad, fn_out='./'):
    import matplotlib.pyplot as plt
    plt.ioff()
    f = plt.figure()
    ax = f.add_subplot(111)
    #for i, val in enumerate(lambdar_arr):
    for i in i_early:
        a = np.asarray(catalog['lambda_arr'][i])
        ax.plot(a, 'r-', alpha=0.5) # Red = Early
    for i in i_late:
        ax.plot(catalog['lambda_arr'][i], 'b-', alpha=0.3) # Red = Early
    
    #plt.xlabel() # in the unit of Reff
    ax.set_title(r"$\lambda _{R}$") 
    ax.set_ylabel(r"$\lambda _{R}$") 
    ax.set_xlabel("["+ r'$R/R_{eff}$'+"]")
    ax.set_xlim(right=9)
    ax.set_ylim([0,1])
    ax.set_xticks([0, 4.5, 9])
    ax.set_xticklabels(["0", "0.5", "1"])
    plt.savefig(fn_out)
    plt.close()
    
def aexp2zred(aexp):
    return [1.0/a - 1.0 for a in aexp]

def zred2aexp(zred):
    return [1.0/(1.0 + z) for z in zred]

def lbt2aexp(lts):
    import astropy.units as u
    from astropy.cosmology import WMAP7, z_at_value
    zreds = [z_at_value(WMAP7.lookback_time, ll * u.Gyr) for ll in lts]
    return [1.0/(1+z) for z in zreds]

def density_map(x, y, ax, sort=True):
    from scipy.stats import gaussian_kde
    xy = np.vstack([x,y])
    z = gaussian_kde(xy)(xy) 
    z /= max(z)

    idx = z.argsort()    
    xx, yy = x[idx], y[idx]
    z = z[idx]
    
    im = ax.scatter(xx, yy, c=z, s=50, edgecolor='')
    return im

In [2]:
#clusters = [5427, 36415, 10002, 39990, 1605, 36413, 4466, 74010][0:5]
clusters = [5427, 36415, 39990, 1605, 10002, 36413, 4466, 74010][0:1]
# 74010 is unreliable.
# 36413 왜 안 돌아가나..?

#exclude_gals = [[],
#                [],
#                [],
#                [123,155,],
#                [2694,4684,5448,5885,5906,6967,6981,7047,7072,7151,7612],
#                []]
lr_points = 10
points = np.arange(lr_points * 3)

In [3]:
class Galaxy:
    # No data encapsulation, I don't care. (it's not meadatory)
    def __init__(self, cluster, final_id, idx, nout_fi=None, nout_ini=None):
        self.cluster = cluster
        self.final_id = final_id
        self.idx = idx
        self.nout_fi = nout_fi
        self.nout_ini = nout_ini
        self.data = None
        #galaxy.data = list of catalog outputs (x nnouts)
        #self.data = [[]]*(self.nout_fi - self.nout_ini + 1)
        self.nout = [[]]*(self.nout_fi - self.nout_ini + 1)
    
    def add_data(self, dtype):
        if self.nout_fi == None or self.nout_ini == None:
            print("set nout_fi and nout_ini first!")
            return
        else:
            new_types = []
            for item in dtype.descr:
                if item[1] != 'lambda_arr':
                    new_types.append(item)
            b = np.zeros(self.nout_fi - self.nout_ini + 1, dtype=new_types)    
    
    def add_catalog_data(self, catalog, nout):
        if self.data == None:
            self.add_data(catalog.dtype)
        nout_ind = nout - nout_ini
        self.nout[nout_ind] = nout
        self.data[nout_ind] = catalog
        #self.data[nout_ind] = catalog
    

In [4]:
f = plt.figure()
ax = f.add_subplot(111)

nout_ini = 37
nout_fi = 187

#for i, cluster in enumerate(clusters):
cluster = clusters[0]
nouts = range(nout_ini, nout_fi+1)
wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5) + '/'

snout = str(nout_fi).zfill(3)
catalog = load_pickle(wdir + 'catalog_GM/' + 'catalog'+ snout +'.pickle')
idfs = np.asarray(catalog['id'])
idxs = np.asarray(catalog['idx'])
# allgall = list of galaxies
# galaxy = list of catalog outputs (x nnouts)
#allgall = [[]]*len(idfs)

# Load tree

alltrees = ctu.load_tree(wdir, is_gal=True)
main_list=[[]]*len(idfs)
for i, idx in enumerate(idxs):
    main_list[i] = ctu.extract_main_tree(alltrees.data, idx)

Loaded an extended tree


In [5]:
# I can't construct a new structured array with 2D array in it. 
# dtype of lambda_arr is not recognized properly.
# So, separate lambda_arr as a new attribute to galaxy instance.
new_types = []
for item in catalog.dtype.descr:
    if item[1] != 'lambda_arr':
        new_types.append(item)

# initialize data arrays of galaxies.
allgal=[]
for j, (idx, idf) in enumerate(zip(idxs, idfs)):
    allgal.append(Galaxy(cluster, idf, idx, nout_fi=nout_fi, nout_ini=nout_ini))# for idx,idf in zip(idxs, idfs))
    allgal[j].data = np.zeros(nout_fi - nout_ini + 1, new_types)
    allgal[j].lambda_arr = np.zeros((nout_fi - nout_ini + 1, len(catalog['lambda_arr'][0])), dtype=float)

#allgal[j].data = np.zeros(nout_fi - nout_ini + 1, new_types)
#allgal[j].lambda_arr = np.zeros((nout_fi - nout_ini + 1, len(catalog['lambda_arr'][0])), dtype=float)


In [6]:
# from the last snapshot.

#b = np.zeros(self.nout_fi - self.nout_ini + 1, dtype=new_types)    

for i, nout in enumerate(reversed(nouts)):
    catalog = load_pickle(wdir + 'catalog_GM/' + 'catalog'+ str(nout) +'.pickle')
    for i_main, main in enumerate(main_list[1:2]):
        
        # Some galaxies are shorter than others.
        #  -> len(main_list[10]['id']) > len(main_list[11]['id'])
        
        i_now = np.where(main_list[i_main]['nout'] == nout)[0]
        
        if len(i_now) > 0:
            # index of catalog of a galaxy with idx at nout. 
            i_gal = np.where(catalog['idx'] == main_list[i_main]['id'][i_now])[0]
            print(nout, i_gal, len(i_gal))
            for dtype in allgal[j].data.dtype.names:
                #print(dtype)
                #print(catalog[i_gal][dtype])
                allgal[i_main].data[i][dtype] = catalog[i_gal][dtype]
                allgal[i_main].nout[i] = nout
            
            #allgal[i_main].add_catalog_data(catalog[i_gal], nout)
    

187 [0] 1
186 [10] 1
185 [21] 1
184 [0] 1
183 [6] 1
182 [] 0


ValueError: setting an array element with a sequence.

In [180]:
nout_main_list = main_list[0]['nout']
#main_list[0]['id']

In [10]:
allgal[0].nout

[187,
 186,
 185,
 184,
 183,
 182,
 181,
 180,
 179,
 178,
 177,
 176,
 175,
 174,
 173,
 172,
 171,
 170,
 169,
 168,
 167,
 166,
 165,
 164,
 163,
 162,
 161,
 160,
 159,
 158,
 157,
 156,
 155,
 154,
 153,
 152,
 151,
 150,
 149,
 148,
 147,
 146,
 145,
 144,
 143,
 142,
 141,
 140,
 139,
 138,
 137,
 136,
 135,
 134,
 133,
 132,
 131,
 130,
 129,
 128,
 127,
 126,
 125,
 124,
 123,
 122,
 121,
 120,
 119,
 118,
 117,
 116,
 115,
 114,
 113,
 112,
 111,
 110,
 109,
 108,
 107,
 106,
 105,
 104,
 103,
 102,
 101,
 100,
 99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 76,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37]

In [18]:
allgal[igal].data['mstar']

array([  3.73028492e+010,   3.73483344e+010,   3.71879878e+010,
         3.71930417e+010,   3.73386860e+010,   1.32554656e+010,
         3.72003929e+010,   3.72137168e+010,   3.72752825e+010,
         3.73621177e+010,   3.73419021e+010,   7.44853237e+009,
         1.32720056e+010,   7.81057552e+006,   3.73437399e+010,
         3.75022486e+010,   3.53773127e+007,   6.92701042e-310,
         3.03234109e+007,   6.92701042e-310,   3.75265993e+010,
         3.76795947e+010,   6.92701042e-310,   3.73308754e+010,
         3.74912220e+010,   3.76451362e+010,   6.92701042e-310,
         3.73451182e+010,   3.74411424e+010,   3.76001106e+010,
         3.75707060e+010,   3.73951978e+010,   4.18095513e+007,
         3.21611933e+007,   3.73814144e+010,   3.71429622e+010,
         6.61601691e+007,   3.71617994e+010,   2.57289547e+007,
         1.30294183e+010,   3.69945612e+010,   6.92701042e-310,
         4.54851163e+007,   1.88372704e+007,   3.70639375e+010,
         1.06591384e+008,   1.07510275e+

In [11]:
igal = 0
plt.plot(allgal[igal].nout, allgal[igal].data[:]['mstar'])
plt.show()

In [4]:
    
    i_early = np.where((catalog['mstar'] > 1e10) & (catalog['mstar'] < 1e11))[0]
    i_late = []
    #i_cluster = np.where(catalog['
#    i_bad = np.where(catalog.id == 0)[0]
#    i_bad = np.concatenate((i_bad, np.where(catalog['lambda_arr'][0] > 0.3)[0]))
    #i_bad = np.concatenate(i_bad, exclude_gals[i])
    n_good += len(i_early)
    for ij, j in enumerate(i_early):
        all_lr = np.concatenate((all_lr, catalog['lambda_arr'][j][0: 2*lr_points])) # some catalog has L_arr up to 2Rvir. 
        xpos.append(catalog['xc'][j])
        ypos.append(catalog['yc'][j])
        zpos.append(catalog['zc'][j])
        clur.append(catalog['xc'][j])
        xpos = xpos[ij] = catalog['xc'][j]
        xpos = xpos[ij] = catalog['xc'][j]
        xpos = xpos[ij] = catalog['xc'][j]


                                 

TypeError: 'numpy.float64' object does not support item assignment

In [9]:
xpoints = np.tile(points, n_good)

im = density_map(xpoints, all_lr.ravel(), ax)
ax.set_xlim([-0.5,2*lr_points])
ax.set_ylim([0,0.8])
x_tick_pos = ""
ax.set_xticks([0,lr_points-1,2*lr_points - 1])
x_ticks_label = ["0", "1", "2"] # Correct. by default, rscale_lambda = 2.0
ax.set_xticklabels(labels = [z for z in x_ticks_label])
ax.set_xlabel(r"$R/R_{eff}$")

#plot_lambda(catalog, i_early, i_late, i_bad, fn_out = str(cluster) + "_lambdar_disk.png")
plt.show()

In [17]:
lr_points

15

In [24]:
print(points)
print(catalog['lambda_arr'][j])#[ :2 *lr_points])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29]
[ 0.09013026  0.15048778  0.17486082  0.17904494  0.20769836  0.22743054
  0.23884647  0.22560917  0.24075304  0.25897361  0.24736996  0.24396904
  0.24362423  0.28476168  0.24230847  0.26043998  0.25583183  0.27019347
  0.29174383  0.27495197]


In [25]:
# as a line
plt.close()

fig, ax = plt.subplots(len(clusters))

for i, cluster in enumerate(clusters):
    wdir = '/home/hoseung/Work/data/' + str(cluster).zfill(5)
    catalog = load_pickle(wdir + '/catalog_GM/' + 'catalog187.pickle')
    #i_early = np.where(catalog['mstar'] > 5e11)[0]
    i_early = np.where((catalog['mstar'] > 1e10) & (catalog['mstar'] < 1e11))[0]
    
    for j in i_early:
        ax[i].plot(points, catalog['lambda_arr'][j][ :2 *lr_points], c='grey', alpha=0.3)
    
    ax[i].set_xlim([-0.5,2*lr_points])
    ax[i].set_ylim([0,0.8])
    x_tick_pos = ""
    ax[i].set_xticks([0,lr_points -1, 2*lr_points - 1])
    x_ticks_label = ["0", "1", "2"] # Correct. by default, rscale_lambda = 2.0
    ax[i].set_xticklabels(labels = [z for z in x_ticks_label])
    ax[i].set_xlabel(r"$R/R_{eff}$")
    
plt.show()    

In [11]:
len(catalog['lambda_arr'][j])

45

In [ ]:
final_gals = list(cat['final_gal'])

    # exclude disky galaxies
for bad_gal in exclude_gals[i]:
    final_gals.remove(bad_gal)

ngals = len(final_gals)
mstar = np.zeros((ngals, nnouts))
l_r = np.zeros((ngals, nnouts))